Import the dependencies and dataset from Kaggle 

In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download victorsoeiro/manga-manhwa-and-manhua-dataset

!mkdir Dataset
!cp /content/manga-manhwa-and-manhua-dataset.zip /content/Dataset/manga-manhwa-and-manhua-dataset.zip
!unzip -q /content/Dataset/manga-manhwa-and-manhua-dataset.zip -d /content/Dataset
!rm /content/Dataset/manga-manhwa-and-manhua-dataset.zip



cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
cp: cannot stat '/content/manga-manhwa-and-manhua-dataset.zip': No such file or directory
unzip:  cannot find or open /content/Dataset/manga-manhwa-and-manhua-dataset.zip, /content/Dataset/manga-manhwa-and-manhua-dataset.zip.zip or /content/Dataset/manga-manhwa-and-manhua-dataset.zip.ZIP.
rm: cannot remove '/content/Dataset/manga-manhwa-and-manhua-

# Preparing the dataset

In [ ]:
!pip install streamlit
import streamlit
import numpy as np
import pandas as pd
import re
!pip install transformers
!pip install torch
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df = pd.read_csv('/content/Dataset/data.csv')

In [ ]:
df = df.drop(df.loc[:, ['rating','year','cover']].columns, axis=1)

In [ ]:
df = df[df['tags'].str.contains('manhwa', case=False)]

In [ ]:
df_main_w_tags = df.reset_index(drop=True)
#now dropping tags, but still has some titles with no description
df_nulldes = df_main_w_tags.drop(df_main_w_tags.loc[:,['tags']], axis=1)

In [ ]:
#no more empty description
mask = df_nulldes['description'].fillna('').str.contains("this entry currently doesn't have a synopsis")
df = df_nulldes[~mask].reset_index(drop=True)
df.dropna(subset=['description'], inplace=True)


In [ ]:
print(df.head())

                       title  \
0     Who Made Me a Princess   
1  Omniscient Reader (Novel)   
2          Omniscient Reader   
3      Tower of God - Part 3   
4                    Bastard   

                                         description  
0  The beautiful Athanasia was killed at the hand...  
1  One day, our MC finds himself stuck in the wor...  
2  Back then, Dokja had no idea. He had no idea h...  
3                  The third season of Tower of God.  
4  Jin has lived a tough life. One of his oldest ...  


In [ ]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds = ds.remove_columns(['__index_level_0__'])

In [ ]:
# Split dataset into train and test sets
dds = ds.train_test_split(0.25, seed=42)
train_ds = dds['train']
test_ds = dds['test']

In [ ]:
dds

DatasetDict({
    train: Dataset({
        features: ['title', 'description'],
        num_rows: 5229
    })
    test: Dataset({
        features: ['title', 'description'],
        num_rows: 1744
    })
})

In [ ]:
dds['train'][0]

{'title': 'Give to the Heart - Memories',
 'description': "Brilliant scientists have always existed ahead of their time. Just give them the necessary resources, and they create miracles. On the colonized world of Mars, Dr. Ano, her assistant Chio, and their friend Ganok existed as gods, joined as one in a technological union with the supercomputer Margot. They formed a trio of immortal Demon Kings who determined the fate of all life on that planet. But long ago, they lived as normal humans on Earth. They went to school together and bonded through shared hardships. They even experienced love. All the while, the powers-that-be kept a nervous eye on Ano's dangerous research. If left unchecked, her research could destroy human civilization."}

# Training the model

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling, TextDataset
from datasets import load_dataset

# Load the dataset
dataset = dds

# Instantiate the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token


def preprocess_function(examples):
    # Tokenize titles and descriptions separately
    titles = examples['title']
    descriptions = examples['description']
    encodings = tokenizer(titles, descriptions, truncation=True, padding=True)

    # Rename the target key to match the model's expected input
    inputIDlist = np.array(encodings['input_ids'])
    attmask = np.array(encodings['attention_mask'])
    encodings['input_ids'] = torch.tensor(inputIDlist[:, :tokenizer.model_max_length])
    encodings['attention_mask'] = torch.tensor(attmask[:, :tokenizer.model_max_length])
    encodings['decoder_input_ids'] = torch.tensor(encodings['input_ids'].clone())

    # Shift decoder_input_ids to the right by one
    # and set the rightmost token to padding
    for i in range(len(encodings['decoder_input_ids'])):
        encodings['decoder_input_ids'][i][:-1] = encodings['decoder_input_ids'][i][1:].clone()
        encodings['decoder_input_ids'][i][-1] = tokenizer.pad_token_id

    return encodings


In [ ]:
train_dataset = dds['train']
test_dataset = dds['test']

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir='results',
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_total_limit=2,
    learning_rate=8e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    fp16=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=test_dataset,
)

trainer.train()

Map:   0%|          | 0/5229 [00:00<?, ? examples/s]

<ipython-input-16-d14170cf3a0e>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  encodings['decoder_input_ids'] = torch.tensor(encodings['input_ids'].clone())


Map:   0%|          | 0/1744 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss
500,3.589500,3.361467
1000,3.248200,3.324524
1500,3.081400,3.327777


TrainOutput(global_step=1962, training_loss=3.2239079995500446, metrics={'train_runtime': 833.1602, 'train_samples_per_second': 18.828, 'train_steps_per_second': 2.355, 'total_flos': 2789152609536000.0, 'train_loss': 3.2239079995500446, 'epoch': 3.0})

In [ ]:
#save trained model and tokenizer to disk:
model.save_pretrained("/content/models/webtoon_model")
tokenizer.save_pretrained("/content/models/webtoon_tokenizer")


('/content/models/webtoon_tokenizer/tokenizer_config.json',
 '/content/models/webtoon_tokenizer/special_tokens_map.json',
 '/content/models/webtoon_tokenizer/vocab.json',
 '/content/models/webtoon_tokenizer/merges.txt',
 '/content/models/webtoon_tokenizer/added_tokens.json')

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Device:", device)


def generate_description(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    # Generate the output using the model
    output = model.generate(
        input_ids=input_ids,
        max_length=256,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=2
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)
print(generate_description("The Omnipresent dragon king returns after 2000 years"))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Device: cuda
The Omnipresent dragon king returns after 2000 years, and he’s determined to become the king of dragons. But he has no idea that the dragons are actually his own creations. Now, he must find a way to stop the dragon from devouring the human heart. Will he be able to do that? Or will he have to use his powers to his advantage? Find out in this thrilling new series of stories about dragons and humans, as they struggle to find their place in the modern world. Watch as dragons battle to reclaim their rightful place as the rulers of the land, or face a new dragon that threatens to devour them all! This is the story of a dragon and a human. Follow the adventures of Jason and Eunseong as they navigate their way through the ups and downs of their daily lives, from the mundane to the fantastical. And what will be their fate? What will happen to their dragons? And will they ever return to being human again?! Will they find love or love? Will they be reborn as a different species? Wh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir "/content/drive/My Drive/models"

mkdir: cannot create directory ‘/content/drive/My Drive/models’: File exists


In [ ]:
!cp -r "/content/models/webtoon_model" "/content/drive/My Drive/models/"

In [ ]:
!cp -r "/content/models/webtoon_tokenizer" "/content/drive/My Drive/models/"